# STA 141B: Homework 5
Winter 2018

## Student Information

After the colons (in the same line) please write just your first name, last name, and the 9 digit student ID number below.

First Name: Julia

Last Name: Stelman

Student ID: 912211657

## Instructions

We use a script that extracts your answers by looking for cells in between the cells containing the exercise statements.  So you 

- MUST add cells in between the exercise statements and add answers within them and
- MUST NOT modify the existing cells, particularly not the problem statement

To make markdown, please switch the cell type to markdown (from code) - you can hit 'm' when you are in command mode - and use the markdown language.  For a brief tutorial see: https://daringfireball.net/projects/markdown/syntax

# Housing data

In this assignment, you'll analyze a collection of data sets from the [San Francisco Open Data Portal](http://data.sfgov.org/) and [Zillow](https://www.zillow.com/). The data sets have been stored in the SQLite database `sf_data.sqlite`, which you can [download here](http://anson.ucdavis.edu/~nulle/sf_data.sqlite). The database contains the following tables:

Table                   | Description
----------------------- | -----------
`crime`                 | Crime reports dating back to 2010.
`mobile_food_locations` | List of all locations where mobile food vendors sell.
`mobile_food_permits`   | List of all mobile food vendor permits. More details [here](https://data.sfgov.org/api/views/rqzj-sfat/files/8g2f5RV4PEk0_b24iJEtgEet9gnh_eA27GlqoOjjK4k?download=true&filename=DPW_DataDictionary_Mobile-Food-Facility-Permit.pdf).
`mobile_food_schedule`  | Schedules for mobile food vendors.
`noise`                 | Noise complaints dating back to August 2015.
`parking`               | List of all parking lots.
`parks`                 | List of all parks.
`schools`               | List of all schools.
`zillow`                | Zillow rent and housing statistics dating back to 1996. More details [here](https://www.zillow.com/research/data/).

The `mobile_food_` tables are explicitly connected through the `locationid` and `permit` columns. The other tables are not connected, but you may be able to connect them using dates, latitude/longitude, or postal codes.

Shapefiles for US postal codes are available [here](https://www.census.gov/geo/maps-data/data/cbf/cbf_zcta.html). These may be useful for converting latitude/longitude to postal codes.

Shapefiles for San Francisco Neighborhoods are available [here](https://data.sfgov.org/Geographic-Locations-and-Boundaries/SF-Find-Neighborhoods/pty2-tcw4).

__Exercise 1.1.__ (20 pts) Which mobile food vendor(s) sells at the most locations?

In [1]:
import geopandas 

In [106]:
import sqlite3 as sql
import pandas as pd

In [3]:
db = sql.connect('sf_data.sqlite')

In [108]:
qq = """
SELECT DISTINCT A.Applicant AS Vendor, B.locationid AS Location
FROM mobile_food_schedule AS B LEFT JOIN mobile_food_permits AS A
ON A.permit = B.permit
"""
print(pd.read_sql(qq,db).shape)
tab1 = pd.read_sql(qq,db)

(679, 2)


In [129]:
VendorsQ = tab1.Vendor.unique()
VQ = list(VendorsQ)
VC = []
list(tab1.Vendor)
for vendor in VendorsQ:
    VC.append(list(tab1.Vendor).count(vendor))

counts = pd.DataFrame(list(zip(VQ,VC)))
counts.columns = ["vendor","num_locations"]
counts[counts.num_locations == counts.num_locations.max()]

,vendor,num_locations
124,May Catering,58


May Catering sells at the most locations (58 locations in total).

 ##
 ##
 ##
 ##
 ##
 ##
 ##
 ##
 ##
 ##

In [24]:
pd.read_sql("SELECT DISTINCT * FROM mobile_food_permits LIMIT 10",db)

,permit,Status,Applicant,FacilityType,FoodItems,PriorPermit,Approved,Expiration
0,16MFF-0027,APPROVED,F & C Catering,Truck,Cold Truck: Hot/Cold Sandwiches: Water: Soda: ...,1,2016-03-09 12:00:00,2017-03-15 12:00:00
1,13MFF-0102,EXPIRED,Natan's Catering,Truck,Burgers: melts: hot dogs: burritos:sandwiches:...,1,2013-04-12 12:00:00,2014-03-15 12:00:00
2,16MFF-0069,APPROVED,Munch A Bunch,Truck,Cold Truck: packaged sandwiches: pitas: breakf...,1,2016-03-15 12:00:00,2017-03-15 12:00:00
3,16MFF-0051,APPROVED,Park's Catering,Truck,Cold Truck: Hamburger: cheeseburgers: hot dogs...,1,2016-03-15 12:00:00,2017-03-15 12:00:00
4,16MFF-0010,REQUESTED,Pipo's Grill,Truck,Tacos: Burritos: Hot Dogs: and Hamburgers,0,None,None
5,16MFF-0024,APPROVED,"Off the Grid Services, LLC",Truck,Everything,0,2016-03-17 12:00:00,2017-03-15 12:00:00
6,14MFF-0034,EXPIRED,Bach Catering,Truck,Cold Truck: Cheeseburgers: Burgers: Chicken Ba...,1,2014-03-13 12:00:00,2015-03-15 12:00:00
7,16MFF-0048,REQUESTED,Singh Brothers Ice Cream,Truck,Ice Cream: Pre-Packaged Chips: Candies: Bottle...,1,None,2017-03-15 12:00:00
8,16MFF-0036,APPROVED,Roadside Rotisserie Corporation / Country Grill,Truck,Rotisserie Chicken: Ribs: Kickass Salad: Potat...,0,2016-03-10 12:00:00,2017-03-15 12:00:00
9,14MFF-0001,EXPIRED,Sun Rise Catering,Truck,Cold Truck: sandwiches: drinks: snacks: candy:...,1,2014-09-10 12:00:00,2015-03-15 12:00:00


In [6]:
pd.read_sql("SELECT * FROM crime WHERE dayofweek is 'Sunday' LIMIT 10",db)

,IncidntNum,Category,Descript,DayOfWeek,Datetime,PdDistrict,Resolution,Address,Lon,Lat
0,150098210,ROBBERY,"ROBBERY, BODILY FORCE",Sunday,2015-02-01 15:45:00,TENDERLOIN,NONE,300 Block of LEAVENWORTH ST,-122.414406,37.784191
1,150098210,ASSAULT,AGGRAVATED ASSAULT WITH BODILY FORCE,Sunday,2015-02-01 15:45:00,TENDERLOIN,NONE,300 Block of LEAVENWORTH ST,-122.414406,37.784191
2,150098210,SECONDARY CODES,DOMESTIC VIOLENCE,Sunday,2015-02-01 15:45:00,TENDERLOIN,NONE,300 Block of LEAVENWORTH ST,-122.414406,37.784191
3,150098232,NON-CRIMINAL,AIDED CASE -PROPERTY FOR DESTRUCTION,Sunday,2015-02-01 16:21:00,RICHMOND,NONE,400 Block of LOCUST ST,-122.451782,37.787085
4,150098345,LARCENY/THEFT,PETTY THEFT SHOPLIFTING,Sunday,2015-02-01 14:00:00,MISSION,"ARREST, BOOKED",1700 Block of HARRISON ST,-122.413354,37.769075
5,150098345,DRUG/NARCOTIC,POSSESSION OF METH-AMPHETAMINE,Sunday,2015-02-01 14:00:00,MISSION,"ARREST, BOOKED",1700 Block of HARRISON ST,-122.413354,37.769075
6,150098345,DRUG/NARCOTIC,POSSESSION OF NARCOTICS PARAPHERNALIA,Sunday,2015-02-01 14:00:00,MISSION,"ARREST, BOOKED",1700 Block of HARRISON ST,-122.413354,37.769075
7,150098345,WARRANTS,WARRANT ARREST,Sunday,2015-02-01 14:00:00,MISSION,"ARREST, BOOKED",1700 Block of HARRISON ST,-122.413354,37.769075
8,150098367,ROBBERY,"ROBBERY, ARMED WITH A KNIFE",Sunday,2015-02-01 16:20:00,MISSION,NONE,2200 Block of MARKET ST,-122.432198,37.765111
9,150098373,VEHICLE THEFT,STOLEN AUTOMOBILE,Sunday,2015-02-01 12:45:00,NORTHERN,NONE,FRANKLIN ST / TURK ST,-122.422277,37.781753


__Exercise 1.2.__ (40 pts) Ask and use the database to analyze 5 questions about San Francisco. For each question, write at least 150 words and support your answer with plots. Make a map for at least 2 of the 5 questions.

You should try to come up with some questions on your own, but these are examples of reasonable questions:

* Which parts of the city are the most and least expensive?
* Which parts of the city are the most dangerous (and at what times)?
* Are noise complaints and mobile food vendors related?
* What are the best times and places to find food trucks?
* Is there a relationship between housing prices and any of the other tables?

Please make sure to clearly state each of your questions in your submission.

Where, if anywhere, do the school vendors sell on weekends?
What crimes are most common around schools?
What types of crimes happen near parking lots? near parks?
What can we infer from the data about food vendor hours base on location? (if near a school, maybe earlier, if near noise complaint hotspot, maybe more late)
What areas have the widest varieties of food vendors coming in?
Is real estate price and crime rate related (obviously)?


In [134]:
pd.read_sql("SELECT * FROM sqlite_master", db)

,type,name,tbl_name,rootpage,sql
0,table,crime,crime,2,"CREATE TABLE ""crime"" (\n""IncidntNum"" INTEGER,\..."
1,table,noise,noise,35775,"CREATE TABLE ""noise"" (\n""CaseID"" INTEGER,\n ""..."
2,table,parking,parking,35921,"CREATE TABLE ""parking"" (\n""Owner"" TEXT,\n ""Ad..."
3,table,schools,schools,35944,"CREATE TABLE ""schools"" (\n""Name"" TEXT,\n ""Ent..."
4,table,parks,parks,35961,"CREATE TABLE ""parks"" (\n""Name"" TEXT,\n ""Type""..."
5,table,zillow,zillow,35967,"CREATE TABLE ""zillow"" (\n""RegionName"" INTEGER,..."
6,table,mobile_food_permits,mobile_food_permits,36050,"CREATE TABLE ""mobile_food_permits"" (\n""permit""..."
7,table,mobile_food_locations,mobile_food_locations,36060,"CREATE TABLE ""mobile_food_locations"" (\n""locat..."
8,table,mobile_food_schedule,mobile_food_schedule,36079,"CREATE TABLE ""mobile_food_schedule"" (\n""locati..."


In [149]:
crime_list = pd.read_sql("SELECT * FROM sqlite_master", db).iloc[0]['sql'][13:]
noise_list = pd.read_sql("SELECT * FROM sqlite_master", db).iloc[1]['sql'][13:]
parking_list = pd.read_sql("SELECT * FROM sqlite_master", db).iloc[2]['sql'][13:]
school_list = pd.read_sql("SELECT * FROM sqlite_master", db).iloc[3]['sql'][13:]
park_list = pd.read_sql("SELECT * FROM sqlite_master", db).iloc[4]['sql'][13:]
zillow_list = pd.read_sql("SELECT * FROM sqlite_master", db).iloc[5]['sql'][13:]
mfp_list = pd.read_sql("SELECT * FROM sqlite_master", db).iloc[6]['sql'][13:]
mfl_list = pd.read_sql("SELECT * FROM sqlite_master", db).iloc[7]['sql'][13:]
mfs_list = pd.read_sql("SELECT * FROM sqlite_master", db).iloc[8]['sql'][13:]
print(crime_list + '\n\n', noise_list + '\n\n', parking_list + '\n\n', school_list + '\n\n', park_list + '\n\n', 
      zillow_list + '\n\n', mfp_list + '\n\n', mfl_list + '\n\n',mfs_list + '\n\n')

"crime" (
"IncidntNum" INTEGER,
  "Category" TEXT,
  "Descript" TEXT,
  "DayOfWeek" TEXT,
  "Datetime" TIMESTAMP,
  "PdDistrict" TEXT,
  "Resolution" TEXT,
  "Address" TEXT,
  "Lon" REAL,
  "Lat" REAL
)

 "noise" (
"CaseID" INTEGER,
  "Type" TEXT,
  "Address" TEXT,
  "Neighborhood" TEXT,
  "Datetime" TIMESTAMP,
  "Lat" REAL,
  "Lon" REAL
)

 "parking" (
"Owner" TEXT,
  "Address" TEXT,
  "PrimeType" TEXT,
  "SecondType" TEXT,
  "GarOrLot" TEXT,
  "RegCap" INTEGER,
  "ValetCap" INTEGER,
  "MCCap" INTEGER,
  "LandType" TEXT,
  "Lat" REAL,
  "Lon" REAL
)

 "schools" (
"Name" TEXT,
  "Entity" TEXT,
  "LowerGrade" INTEGER,
  "UpperGrade" INTEGER,
  "GradeRange" TEXT,
  "Category" TEXT,
  "LowerAge" INTEGER,
  "UpperAge" INTEGER,
  "GeneralType" TEXT,
  "Address" TEXT,
  "Lat" REAL,
  "Lon" REAL
)

 "parks" (
"Name" TEXT,
  "Type" TEXT,
  "Acreage" REAL,
  "ParkID" INTEGER,
  "Lat" REAL,
  "Lon" REAL
)

 "zillow" (
"RegionName" INTEGER,
  "Date" TIMESTAMP,
  "ZriPerSqft_AllHomes" REAL,
  "Med

In [69]:
q = """
SELECT A.Applicant AS Vendor, B.locationid AS Location, C.latitude, C.longitude
FROM mobile_food_schedule AS B LEFT JOIN mobile_food_locations AS C LEFT JOIN mobile_food_permits AS A
ON A.permit = B.permit AND B.locationid = C.locationid
LIMIT 10
"""
pd.read_sql(q,db)

In [65]:
r = """
SELECT COUNT(*) AS len_mfp
FROM mobile_food_permits AS A
"""
s = """
SELECT COUNT(*) AS len_mfs
FROM mobile_food_schedule AS B
"""
t = """
SELECT COUNT(*) AS len_mfp
FROM mobile_food_locations AS C
"""
df1 = pd.read_sql(r,db)
df2 = pd.read_sql(s,db)
df3 = pd.read_sql(t,db)

In [ ]:
db.close()